In [1]:
from tool import *
import plotly.express as px
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)
with open('D:/Data/ODM PO.db', 'rb') as f:
    po = pickle.load(f)
    po = po[po['Cancel'] == 'N']
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr_db = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    ste = pickle.load(f)

In [3]:
spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-16:, ] # 선택할 SP의 Version 목록을 출력

,Planweek,Category,Created_at,SP
134,2024-02-12(W07),Latest26w,202402121709,1129621.0
135,2024-02-12(W07),Latest26w,202402130921,1130892.0
136,2024-02-12(W07),Latest26w,202402150843,1134445.0
137,2024-02-19(W08),Final26w,202402201401,1190717.0
138,2024-02-19(W08),Final52w,202402210937,1432106.0
139,2024-02-19(W08),Latest26w,202402191522,1191421.0
140,2024-02-19(W08),Latest26w,202402200910,1199002.0
141,2024-02-19(W08),Latest26w,202402221037,1190977.0
142,2024-02-26(W09),Final26w,202402271452,1242570.0
143,2024-02-26(W09),Final52w,202402281001,1467368.0


In [4]:
i1 = 146 # 추출할 SP를 선택

p_wk = spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Planweek']
sp = spdb.merge(spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i1], :3])
sp['Month'] = sp['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)

# IT ODM 모델명에서 시리즈명 추출
c1 = sp['From Site'].str.contains('AUO|TPV|TCL|BOEVT') # 모니터 ODM 일 경우
c2 = sp['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sp['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우

sp.loc[c1, 'Series'] = sp[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명은 Inch로 함
sp.loc[c2, 'Series'] = sp[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '', regex=True).apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sp.loc[c3, 'Series'] = 'CL600'

sp = pd.merge(sp, site[['To Site', 'Subsidiary', 'Country']], how='left')

# SCM/MKT 담당자명 Mapping
# cont['Subsidiary'] = cont['Subsidiary'].str.split(e|xpand=True)[0]
sp.loc[sp['Division']=='PC', 'SCM'] = sp.loc[sp['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='PC', 'MKT'] = sp.loc[sp['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='MNT', 'SCM'] = sp.loc[sp['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['Division']=='MNT', 'MKT'] = sp.loc[sp['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
us_b2b = sp.loc[sp['To Site'].str.contains('ENUS') & (sp['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sp.loc[sp['To Site'].str.contains('ENUS') & (sp['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b
sp['Region'] = sp['Region'].replace({'EU':'유럽', 'KOREA':'한국', 'NA':'북미', 'INDIA':'인도', 'MEA':'중아', 'SCA':'중남미', 'SEA':'아시아', 'CHINA':'중국'})
sp['Division'] = sp['Division'].replace({'MNT':'모니터'})
sp['From Site'] = sp['From Site'].replace({'OS_QUANTA_CN_P':'QUANTA', 'OS_AUO_CN_P':'AUO', 'OS_TCL_CN_P':'TCL', 'OS_TPV_CN_P':'TPV', 'OS_PEGATRON_CN_P':'PEGATRON','OS_WINGTECH_CN_P':'WINGTECH', 'OS_BOEVT_CN_P':'BOEVT'})
sp = sp.rename(columns={'Division':'구분', 'From Site':'업체', 'Series':'모델', 'Month':'월', 'Region':'지역', 'Category':'카테고리'})

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_9468\2151161994.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  sp.loc[c2, 'Series'] = sp[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [5]:
end_month = '2024-' + f"{get_month_from_date(datetime.datetime.strptime(p_wk[:10], '%Y-%m-%d'))+2:02d}" # M+2 까지 현황만 Check
starting_month = '2024-'+ '{:02d}'.format(get_month_from_date(datetime.datetime.strptime(p_wk[:10], '%Y-%m-%d')))
starting_month, end_month

('2024-03', '2024-05')

In [6]:
sp = sp[sp['월'] >= starting_month]
sp = sp[sp['월'] <= end_month]

In [7]:
sp['유형'] = 'SP'
sp = sp[~sp['업체'].str.contains('QUANTA|PEGATRON')] # Quanta, Pegatron은 제외해서 작성
sp.loc[sp['업체'].str.contains('WINGTECH'), '구분'] = '태블릿'
sp = pd.concat([sp, sp.iloc[:, :-1]]).fillna('예상').reset_index(drop=True)
sp = sp.drop(columns=['Planweek', 'Created_at', '카테고리', 'Frozen'])
updated_time = datetime.datetime.now().strftime('%y%m%d%H')
filepath = f'D:/주간아웃소싱공급현황/{p_wk}_IT ODM 공급현황_{updated_time}.xlsx'

with pd.ExcelWriter(filepath) as writer:
    sp.to_excel(writer, sheet_name='RAW', index=False, startrow=1)    

In [8]:
wb = load_workbook(filepath)

color = PatternFill(fgColor='BEBEBE', fill_type='solid') # Gray
color2 = PatternFill(fgColor='F5F5DC', fill_type='solid') # Beige
big_size = Font(name='맑은 고딕', size=20, bold=True, color='000000')
middle_size = Font(name='맑은 고딕', size=15, bold=True, color='000099')

border_style = Side(border_style="thin", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)

m = datetime.datetime.strptime(p_wk[:10], '%Y-%m-%d').month
d = datetime.datetime.strptime(p_wk[:10], '%Y-%m-%d').day

sh = wb[wb.sheetnames[0]]
sh['A1'] = f'■ {m}/{d}W 기준 ODM SP Raw 데이터'
sh['A1'].font = middle_size
sh.freeze_panes = 'A3'
for row in sh.iter_rows(min_row=2, min_col=1, max_row=2, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
for row in sh.iter_rows(min_row=3, min_col=1, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'

sh.column_dimensions['E'].width = 20
sh.column_dimensions['F'].width = 20
sh.column_dimensions['G'].width = 15

for name in wb.sheetnames:
    wb[name].sheet_view.showGridLines = False
wb.save(filepath)

In [9]:
print('''
안녕하십니까?

금주 SP 기준으로 ODM 공급 현황 업데이트 부탁 드립니다.
PC는 임승만 책임께서 별도로 자료 송부 부탁 드립니다.
- 회신기한 : 매주 목요일 퇴근전까지

* 작성방법
1) 첨부 파일의 Raw 데이터에서 유형을 "예상"으로 필터링한 후
2) 담당하시는 업체로 필터링한 후,
3) 이번주 Final 기준으로 반영된 SP에서 공급 이슈가 있는 부분에 대해서 가능한 수량이나 가능한 선적 주차로 조정한다.(월이 바뀌면 월도 조정한다)
''')


안녕하십니까?

금주 SP 기준으로 ODM 공급 현황 업데이트 부탁 드립니다.
PC는 임승만 책임께서 별도로 자료 송부 부탁 드립니다.
- 회신기한 : 매주 목요일 퇴근전까지

* 작성방법
1) 첨부 파일의 Raw 데이터에서 유형을 "예상"으로 필터링한 후
2) 담당하시는 업체로 필터링한 후,
3) 이번주 Final 기준으로 반영된 SP에서 공급 이슈가 있는 부분에 대해서 가능한 수량이나 가능한 선적 주차로 조정한다.(월이 바뀌면 월도 조정한다)



## 편집된 엑셀 파일 읽어오기

In [39]:
filepath = get_filename()
p_wk = filepath.split('_')[0].split('/')[-1]
sp = pd.read_excel(filepath, sheet_name='RAW', skiprows=1)

In [40]:
# 업체별 피벗 자료
pv1 = sp.groupby(['구분', '업체', '유형', '월', 'Week Name']).sum('SP').unstack(['유형']).fillna(0)
pv1[('SP', '차이')] = pv1[('SP', '예상')] - pv1[('SP', 'SP')]
pv1 = pv1.stack().reset_index()
# 업체별/모델별 피벗 자료
pv2 = sp.groupby(['구분', '업체', '모델', '유형', '월', 'Week Name']).sum('SP').unstack(['유형']).fillna(0)
pv2[('SP', '차이')] = pv2[('SP', '예상')] - pv2[('SP', 'SP')]
pv2 = pv2.stack().reset_index()

In [41]:
pv1 = pd.concat([pv1, pv1.assign(업체=lambda x:'계')]).reset_index(drop=True)
pv2 = pd.concat([pv2, pv2.assign(모델=lambda x:'계'), pv2.assign(모델=lambda x:'계', 업체=lambda x:'총계')]).reset_index(drop=True)

In [42]:
container1 = []
for i, gr in pv1.groupby('월'):
    a = gr.groupby(['구분', '업체', '유형', '월', 'Week Name']).sum('SP').unstack(['월', 'Week Name'])
    a = a.sort_index(axis=1)
    b = gr.groupby(['구분', '업체', '유형', '월']).sum('SP').unstack('월')
    a.columns = a.columns.droplevel(0)
    b.columns = pd.MultiIndex.from_tuples([(f"{int(i.split('-')[-1])}월", '합계')])
    ab = pd.concat([a, b], axis=1)
    container1.append(ab)
container2 = []
for i, gr in pv2.groupby('월'):
    a = gr.groupby(['업체', '모델', '유형', '월', 'Week Name']).sum('SP').unstack(['월', 'Week Name'])
    a = a.sort_index(axis=1)
    b = gr.groupby(['업체', '모델', '유형', '월']).sum('SP').unstack('월')
    a.columns = a.columns.droplevel(0)
    b.columns = pd.MultiIndex.from_tuples([(f"{int(i.split('-')[-1])}월", '합계')])
    ab = pd.concat([a, b], axis=1)
    container2.append(ab)

In [43]:
with pd.ExcelWriter(filepath) as writer:
    pd.concat(container1, axis=1).sort_index().fillna(0).to_excel(writer, sheet_name='업체별', startrow=1)
    pd.concat(container2, axis=1).sort_index().fillna(0).to_excel(writer, sheet_name='모델별', startrow=1)
    sp.to_excel(writer, sheet_name='RAW', index=False, startrow=1)

In [44]:
m = datetime.datetime.strptime(p_wk[:10], '%Y-%m-%d').month
d = datetime.datetime.strptime(p_wk[:10], '%Y-%m-%d').day

In [45]:
wb = load_workbook(filepath)

color = PatternFill(fgColor='BEBEBE', fill_type='solid') # Gray
color2 = PatternFill(fgColor='F5F5DC', fill_type='solid') # Beige
big_size = Font(name='맑은 고딕', size=20, bold=True, color='000000')
middle_size = Font(name='맑은 고딕', size=15, bold=True, color='000099')

border_style = Side(border_style="thin", color="000000")
border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)

sh = wb[wb.sheetnames[0]]
sh.freeze_panes = 'D5'
sh['A1'] = f'■ {m}/{d}W 기준 모니터/태블릿 M+2 공급 현황'
sh['A1'].font = middle_size

for row in sh.iter_rows(min_row=2, min_col=3, max_row=3, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

for row in sh.iter_rows(min_row=4, min_col=1, max_row=4, max_col=3):
    for cell in row:
        cell.fill = color
        
for row in sh.iter_rows(min_row=5, min_col=3):
        if row[0].value == '차이':
            for cell in row:
                cell.fill = color2

for row in sh.iter_rows(min_row=5, min_col=4, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.column_dimensions['B'].width = 12

sh[f'A{sh.max_row+2}'] = '● 업체별 이슈 사항'

sh = wb[wb.sheetnames[1]]
sh['A1'] = f'■ {m}/{d}W 기준 업체별/모델별 M+2 공급 현황'
sh['A1'].font = middle_size
sh.freeze_panes = 'D5'
for row in sh.iter_rows(min_row=2, min_col=3, max_row=3, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

for row in sh.iter_rows(min_row=4, min_col=1, max_row=4, max_col=3):
    for cell in row:
        cell.fill = color
        
for row in sh.iter_rows(min_row=5, min_col=3):
        if row[0].value == '차이':
            for cell in row:
                cell.fill = color2

for row in sh.iter_rows(min_row=5, min_col=4, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
sh.column_dimensions['A'].width = 12

sh = wb[wb.sheetnames[2]]
sh['A1'] = f'■ {m}/{d}W 기준 ODM SP Raw 데이터'
sh['A1'].font = middle_size
sh.freeze_panes = 'A3'
for row in sh.iter_rows(min_row=2, min_col=1, max_row=2, max_col=sh.max_column):
    for cell in row:
        cell.fill = color
for row in sh.iter_rows(min_row=3, min_col=1, max_row=sh.max_row, max_col=sh.max_column):
    for cell in row:
        cell.border = border
        cell.number_format = '#,##0;[RED]Δ#,##0;"-"'

sh.column_dimensions['E'].width = 20
sh.column_dimensions['F'].width = 20
sh.column_dimensions['G'].width = 17

for name in wb.sheetnames:
    wb[name].sheet_view.showGridLines = False
wb.save(filepath)